# Allele Frequency Calcualtion

This notebook is an introduction to a somewhat primative topic: Calcualting allele frequencies. While these calculations can be done with dedicated packages, we will practice writing out functions to calculate your own.

The only libraries we allow ourselves to use here are `numpy`, `pandas` (mainly for visualization), and `matplotlib`, except for some special util functions later on. Let's start by loading these libraries in as usual.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

seed = 4321
np.random.seed(seed)

## Allele and Genotype Frequencies

A locus is a position in the genome that may contain one or more alleles. The combination of _alleles_ for an individual sample at a locus is known as the _genotype_. For humans, at most location we observe 2 different alleles due to evolutionalry pressures so commonly a di-allelic model (2 known alleles per locus) is used, but the of this exercise will be to evaluate _k-allelic_ loci or loci where any number of alleles can occur.

### Generating Sample Data

To start out we will generate 5 sample genotypes using a tuple to represent the two allels. For now we will assume a biallelic model where each sample only has 2 alleles at each locus. We will also assume that the choices can be `{A,C,T,G,.}` where `.` represents a no-call, or a deletion. Additionally, we will set a probability distribution for each letter's occurrence so that the majority fall within a 2 alleles.

In [2]:
bases = ['A','C','T','G','.']
prob = [.39,.39,.11,.07,.04]
n = 10

In [3]:
genotypes = [tuple(np.random.choice(bases,2,p=prob)) for i in range(n)]
genotypes

[('A', 'T'),
 ('C', 'A'),
 ('A', '.'),
 ('C', 'C'),
 ('A', 'A'),
 ('C', 'C'),
 ('A', 'C'),
 ('C', 'G'),
 ('A', 'C'),
 ('A', 'C')]

### Allele Frequency

The allele frequency from this set now becomes straightforward to calculate when evaluating via brute force.  The frequency will become the the count of the specific Allele's $D$ occurrence over two times the total sample count $N$. We use two times the total sample count since we are using a _bi-allelic_ model.  This frequency can be expresed like this:
$$f(D)=\frac{N_{D}}{2N}$$

In [4]:
def af(genotypes,allele):
    sample_n = len(genotypes)
    allele_ct = 0
    for gt in genotypes:
        allele_ct += gt.count(allele)
    return allele_ct/(2*sample_n) 

In [5]:
[f'{i}: ' + str(af(genotypes,i)) for i in bases]

['A: 0.4', 'C: 0.45', 'T: 0.05', 'G: 0.05', '.: 0.05']

## Genotype Frequencies

The genotype frequency as well can be calculated straight forward from this directly.  The frequency will become the the count of the specific Genotype's $Dd$ occurrence, regardless of order, over the total sample count $N$, like this:
$$f(Dd)=\frac{N_{Dd}}{N}$$

In [6]:
def gf(genotypes,gt):
    sample_n = len(genotypes)
    genotype_ct = genotypes.count(gt)
    genotype_ct += genotypes.count(gt[::-1]) if len(set(gt)) > 1 else 0
    return genotype_ct/(sample_n) 

In [7]:
gt = ('A','C')
f'{gt}:' + str(gf(genotypes,gt))

"('A', 'C'):0.4"

## Using Gentoype Frequency to Calcualte Allele Frequency

The genotype frequency can also be used to calcualte the allele frequency. We can do this by summing the frequency of the homozygous allele with half of the frequency of all the heterozygous alleles. We only use half of the frequncy in heterozygous occurrences since each heterozygous genotype accounts for both the allele of interest and another allele. This can be expressed like this:
$$f(d)=f(dd)+\sum_{j:j\not=d}{\frac{f(dj)}{2}}$$

In [8]:
def af_from_gt(genotypes,allele):
    hom_freq = gf(genotypes,(allele,allele))
    het_freq = 0
    het_genotypes = set([tuple(sorted(i)) for i in genotypes if ('A' in i and len(set(i))>1)])
    for gt in het_genotypes:
        het_freq += gf(genotypes,gt)/2
    return hom_freq + het_freq

In [9]:
allele = 'A'
f'{allele}: ' + str(af_from_gt(genotypes,allele))

'A: 0.4'